<a href="https://colab.research.google.com/github/Sajedah25/Comparing-Arabic-Transformer-Models-for-Question-Answering-Task-NLP-PROJECT-/blob/main/CAMELBERTNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALLATION - IMPORTING**

In [4]:
import json

path = "/content/drive/MyDrive/Colab Notebooks/CAMELBERT (1).ipynb"  # change this
with open(path, "r", encoding="utf-8") as f:
    nb = json.load(f)

if "metadata" in nb and "widgets" in nb["metadata"]:
    nb["metadata"].pop("widgets")

with open(path, "w", encoding="utf-8") as f:
    json.dump(nb, f, ensure_ascii=False, indent=1)

print("Fixed: removed metadata.widgets")



Fixed: removed metadata.widgets


In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install -U transformers
!pip install evaluate
!pip install emoji
!pip install beautifulsoup4 lxml
!pip install arabic-reshaper
!pip install fuzzysearch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.2
    Uninstalling transformers-4.57.2:
      Successfully uninstalled transformers-4.57.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
import collections
import re
import torch
import numpy as np
import evaluate
from transformers import TrainingArguments
from transformers import Trainer, default_data_collator
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from fuzzysearch import find_near_matches
from torch.utils.data import DataLoader

In [ ]:
#ARABIC-SQUAD
arabic_squad = load_dataset("i0xs0/Arabic-SQuAD")
#ARCD
arcd = load_dataset("hsseinmz/arcd")
#AQAD
aqad= load_dataset("arbml/AQAD")
#TYDIQA-GOLDP-ARABIC
tydiqa= load_dataset("asas-ai/tydiqa-goldp-ar")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/355 [00:00<?, ?B/s]

Arabic-SQuAD.json:   0%|          | 0.00/51.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1 [00:00<?, ? examples/s]

README.md: 0.00B [00:00, ?B/s]

...

README.md:   0%|          | 0.00/876 [00:00<?, ?B/s]

data/train-00000-of-00001-dd77b4d20611ad(…):   0%|          | 0.00/8.72M [00:00<?, ?B/s]

data/validation-00000-of-00001-b4cc3a002(…):   0%|          | 0.00/490k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14805 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/921 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-ca")
model = AutoModelForQuestionAnswering.from_pretrained("CAMeL-Lab/bert-base-arabic-camelbert-ca")

tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/468 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**UNIFIYING THE STRUCTURE**

FLATTEN ARABIC SQUAD

In [ ]:
def flatten_arabic_squad_original(dataset):
    flat = []

    # Extract the actual dictionary from the HF Dataset
    sample = dataset[0]
    articles = sample["data"]

    for article in articles:
        for para in article["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                if len(qa["answers"]) == 0:
                    continue

                flat.append({
                    "id": qa["id"],
                    "context": context,
                    "question": qa["question"],
                    "answers": qa["answers"]
                })

    return flat



In [ ]:
flat_squad = flatten_arabic_squad_original(arabic_squad["train"])
flat_squad = Dataset.from_list(flat_squad)


model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

In [ ]:
flat_squad[0]


{'id': '570bce516b8089140040fa42',
 'context': 'يعتمد ASCII أساس ا على الأبجدية الإنجليزية ، ويقوم بترميز 128 حرف ا محدد ا في أعداد صحيحة من سبعة أجزاء كما هو موضح في مخطط ASCII على اليمين . الأحرف المشفرة هي الأرقام من 0 إلى 9 ، والأحرف الصغيرة ا إلى ز ، والأحرف الكبيرة A إلى Z ، ورموز الترقيم الأساسية ، ورموز التحكم التي نشأت مع أجهزة تيليتيبي ، ومساحة . على سبيل المثال ، سيصبح الحرف الصغير ج 1101010 والعشري 106 . تتضمن ASCII تعريفات لـ 128 حرف ا 33 حرف ا تحكم ا غير الطباعة العديد منها الآن قديمة تؤثر على كيفية معالجة النص والمساحة و 95 حرف ا قابلا للطباعة ، بما في ذلك المساحة التي ي عتبر رسم ا غير مرئي 223 .',
 'question': 'ما هو ASCII على أساس؟',
 'answers': [{'answer_start': 23, 'text': 'الأبجدية الإنجليزية'}]}

ARCD AND AQAD

In [ ]:
def unify_arcd_aqad(dataset):
    """Works for both ARCD and AQAD because both have the same flat structure."""
    return dataset.select_columns(["id", "context", "question", "answers"])


In [ ]:
arcd_train = unify_arcd_aqad(arcd["train"])
arcd_valid = unify_arcd_aqad(arcd["validation"])


In [ ]:
print(arcd_train.column_names)
print(arcd_valid.column_names)


['id', 'context', 'question', 'answers']
['id', 'context', 'question', 'answers']


In [ ]:
aqad_train = unify_arcd_aqad(aqad["train"])


In [ ]:
print(aqad_train.column_names)


['id', 'context', 'question', 'answers']


TYDIQA

In [ ]:
def unify_tydiqa(dataset):
    def convert(row):
        ans = row["answers"]

        # Use start_char as answer_start
        answer_start = ans["start_char"] if "start_char" in ans else ans["start_byte"]

        return {
            "id": row["id"],
            "context": row["passage_text"],
            "question": row["question_text"],
            "answers": {
                "text": ans["text"],
                "answer_start": answer_start
            }
        }

    return dataset.map(convert, remove_columns=dataset.column_names)


In [ ]:
tydiqa_train = unify_tydiqa(tydiqa["train"])
tydiqa_valid = unify_tydiqa(tydiqa["validation"])

Map:   0%|          | 0/14805 [00:00<?, ? examples/s]

Map:   0%|          | 0/921 [00:00<?, ? examples/s]

In [ ]:
print(tydiqa_train.column_names)
print(tydiqa_valid.column_names)

['id', 'answers', 'context', 'question']
['id', 'answers', 'context', 'question']


In [ ]:
print("Train size:", len(tydiqa_train))
print("Validation size:", len(tydiqa_valid))

Train size: 14805
Validation size: 921


**PREPROCESSING FROM THE PAPER**

In [ ]:
import re
from fuzzysearch import find_near_matches

# 1. CAMELBERT CLEANING
DIAC = re.compile(r"[ًٌٍَُِّْ]")
TATWEEL = "ـ"

def camelbert_clean(text):
    if text is None:
        return ""
    # Remove diacritics
    text = re.sub(DIAC, "", text)
    # Remove tatweel
    text = text.replace(TATWEEL, "")
    # Normalize whitespace
    text = " ".join(text.split())
    return text


# 2. REALIGN ANSWER AFTER CLEANING
def realign_answer_camel(old_context, new_context, answer_text, old_start):
    cleaned_answer = camelbert_clean(answer_text)

    # Step 1: Exact match
    idx = new_context.find(cleaned_answer)
    if idx != -1:
        return cleaned_answer, idx

    # Step 2: Estimate approximate new start
    before_old = old_context[:old_start]
    before_new = camelbert_clean(before_old)

    removed = len(before_old) - len(before_new)
    est_start = max(0, old_start - removed)

    search_end = min(len(new_context), est_start + len(cleaned_answer) + 50)

    # Step 3: Fuzzy matching for robustness
    matches = find_near_matches(
        cleaned_answer,
        new_context[est_start:search_end],
        max_l_dist=min(8, len(cleaned_answer) // 2)
    )

    if matches:
        m = matches[0]
        return m.matched, est_start + m.start

    # No match found
    return cleaned_answer, -1



# 3. UNIVERSAL PREPROCESSING FUNCTION — CAMELBERT
def preprocess_camelbert_with_alignment(example, tokenizer):
    old_context = example["context"]
    new_context = camelbert_clean(old_context)
    example["context"] = new_context

    # Clean + trim question (no spans → no realignment needed)
    q = camelbert_clean(example["question"])
    q_tokens = tokenizer.tokenize(q)
    q_tokens = q_tokens[:64]  # max question length
    example["question"] = tokenizer.convert_tokens_to_string(q_tokens)

    # Handle answer formats
    answers = example["answers"]

    # FORMAT A — SQuAD list format
    if isinstance(answers, list):
        new_answers = []
        for ans in answers:
            txt = ans["text"]
            start = ans["answer_start"]
            new_text, new_start = realign_answer_camel(old_context, new_context, txt, start)
            new_answers.append({"text": new_text, "answer_start": new_start})
        example["answers"] = new_answers
        return example

    # FORMAT B — ARCD dict format
    if isinstance(answers, dict):
        new_texts = []
        new_starts = []
        for txt, start in zip(answers["text"], answers["answer_start"]):
            new_text, new_start = realign_answer_camel(old_context, new_context, txt, start)
            new_texts.append(new_text)
            new_starts.append(new_start)
        example["answers"] = {
            "text": new_texts,
            "answer_start": new_starts
        }
        return example

    return example



In [ ]:
flat_squad_clean = flat_squad.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning flat SQuAD (CAMeLBERT)"
)


Cleaning flat SQuAD (CAMeLBERT):   0%|          | 0/48344 [00:00<?, ? examples/s]

In [ ]:
arcd_train_clean = arcd_train.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning ARCD Train (CAMeLBERT)"
)


Cleaning ARCD Train (CAMeLBERT):   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
arcd_valid_clean = arcd_valid.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning ARCD Valid (CAMeLBERT)"
)


Cleaning ARCD Valid (CAMeLBERT):   0%|          | 0/702 [00:00<?, ? examples/s]

In [ ]:
aqad_train_clean = aqad_train.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning AQAD Train (CAMeLBERT)"
)

Cleaning AQAD Train (CAMeLBERT):   0%|          | 0/17911 [00:00<?, ? examples/s]

In [ ]:
tydiqa_train_clean = tydiqa_train.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning TyDiQA Train (CAMeLBERT)"
)

Cleaning TyDiQA Train (CAMeLBERT):   0%|          | 0/14805 [00:00<?, ? examples/s]

In [ ]:
tydiqa_valid_clean = tydiqa_valid.map(
    lambda x: preprocess_camelbert_with_alignment(x, tokenizer),
    desc="Cleaning TyDiQA Valid (CAMeLBERT)"
)

Cleaning TyDiQA Valid (CAMeLBERT):   0%|          | 0/921 [00:00<?, ? examples/s]

**TRAINING PREPROCESSING FROM HF**

In [ ]:
max_length = 384
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        ans = answers[sample_idx]

        # CASE 1 — SQuAD format (list of dicts)
        if isinstance(ans, list):
            if len(ans) == 0:
                # No answer → label is (0, 0)
                start_positions.append(0)
                end_positions.append(0)
                continue

            ans = ans[0]  # first answer
            text = ans["text"]
            start_char = ans["answer_start"]

        # CASE 2 — ARCD / AQAD / TydiQA format (dict of lists)
        elif isinstance(ans, dict):
            if len(ans["text"]) == 0:
                # No answer → label is (0, 0)
                start_positions.append(0)
                end_positions.append(0)
                continue

            text = ans["text"][0]
            start_char = ans["answer_start"][0]

        # Compute end character
        end_char = start_char + len(text)

        sequence_ids = inputs.sequence_ids(i)

        # find context range
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while idx < len(sequence_ids) and sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # Check if answer is inside context
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # find start token
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            # find end token
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


**VALIDATION PREPROCESSING FROM HF**

In [ ]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

**CAMELBERT ON TYDIQA**

In [ ]:
train_dataset = tydiqa_train_clean.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=tydiqa_train_clean.column_names,
)


Map:   0%|          | 0/14805 [00:00<?, ? examples/s]

In [ ]:
valid_dataset = tydiqa_valid_clean.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=tydiqa_valid_clean.column_names,
)


Map:   0%|          | 0/921 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

/tmp/ipython-input-1673797489.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,5.711400
20,4.688300
30,3.947000
40,3.986200
50,3.522500
60,3.084000
70,3.440100
80,3.025600
90,2.714100
100,3.073300


TrainOutput(global_step=11664, training_loss=1.0080272389680598, metrics={'train_runtime': 1578.3356, 'train_samples_per_second': 29.56, 'train_steps_per_second': 7.39, 'total_flos': 9143296111706112.0, 'train_loss': 1.0080272389680598, 'epoch': 3.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF")
tokenizer.save_pretrained("/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF")

('/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF/tokenizer_config.json',
 '/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF/special_tokens_map.json',
 '/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF/vocab.txt',
 '/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF/added_tokens.json',
 '/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/CAMELBERT_ON_Tydiqa_HF"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
eval_set_for_model = valid_dataset.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

batch = default_data_collator(eval_set_for_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch = {k: v.to(device) for k, v in batch.items()}
model = model.to(device)
with torch.no_grad():
    outputs = model(**batch)

start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()


In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(valid_dataset):
    example_to_features[feature["example_id"]].append(idx)


In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in tydiqa_valid_clean:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = valid_dataset["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]}
    for ex in tydiqa_valid_clean
]


In [ ]:
print(predicted_answers[1])
print(theoretical_answers[1])

{'id': '-3358420169913421088-0', 'prediction_text': 'بيير كوري'}
{'id': '-3358420169913421088-0', 'answers': {'answer_start': [43, 51, 52], 'text': ['الفرنسي (بيير كوري) وأخوه (جاك)', '(بيير كوري) وأخوه (جاك)', 'بيير كوري) وأخوه (جاك']}}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 73.50705754614549, 'f1': 83.81303583643367}

**CAMELBERT ON ARCD**

In [ ]:
arcd_train_dataset = arcd_train_clean.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=arcd_train_clean.column_names,
)

Map:   0%|          | 0/693 [00:00<?, ? examples/s]

In [ ]:
arcd_test_dataset = arcd_valid_clean.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=arcd_valid_clean.column_names,
)

Map:   0%|          | 0/702 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=arcd_train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

/tmp/ipython-input-3131158956.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,4.147000
20,2.786300
30,2.573200
40,2.576100
50,2.508200
60,2.496700
70,2.524800
80,2.289000
90,2.519300
100,2.696700


TrainOutput(global_step=708, training_loss=1.285159227874993, metrics={'train_runtime': 109.4096, 'train_samples_per_second': 25.775, 'train_steps_per_second': 6.471, 'total_flos': 552642640496640.0, 'train_loss': 1.285159227874993, 'epoch': 4.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF")
tokenizer.save_pretrained("/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF")

('/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF/tokenizer_config.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF/special_tokens_map.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF/vocab.txt',
 '/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF/added_tokens.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/CAMELBERT_ON_ARCD_HF"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
eval_set_for_model = arcd_test_dataset.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

batch = default_data_collator(eval_set_for_model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch = {k: v.to(device) for k, v in batch.items()}
model = model.to(device)
with torch.no_grad():
    outputs = model(**batch)

start_logits = outputs.start_logits.cpu().numpy()
end_logits = outputs.end_logits.cpu().numpy()

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(arcd_test_dataset):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in arcd_valid_clean:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = arcd_test_dataset["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]}
    for ex in arcd_valid_clean
]

In [ ]:
print(predicted_answers[1])
print(theoretical_answers[1])

{'id': '189105393656', 'prediction_text': 'حمزة بن عبد المطلب الهاشمي القرشي'}
{'id': '189105393656', 'answers': {'text': ['وخير أعمامي'], 'answer_start': [153]}}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 32.9059829059829, 'f1': 63.48970967177573}

**CAMELBERT ON AQAD**

In [ ]:
def is_valid(example):
    if example["context"] is None or example["context"] == "":
        return False
    if example["question"] is None or example["question"] == "":
        return False
    if example["answers"] is None or len(example["answers"]["text"]) == 0:
        return False
    if example["answers"]["text"][0] in [None, ""]:
        return False
    if example["answers"]["answer_start"][0] is None:
        return False
    return True

aqad_train_clean = aqad_train_clean.filter(is_valid)


Filter:   0%|          | 0/17911 [00:00<?, ? examples/s]

In [ ]:
len(aqad_train_clean)


11491

In [ ]:
aqad_train_final = aqad_train_clean.select(range(4108))
aqad_test_final  = aqad_train_clean.select(range(4108, 4108+1151))


In [ ]:
aqad_train_dataset = aqad_train_final.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=aqad_train_final.column_names,
)

Map:   0%|          | 0/4108 [00:00<?, ? examples/s]

In [ ]:
aqad_test_dataset = aqad_test_final.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=aqad_test_final.column_names,
)

Map:   0%|          | 0/1151 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=aqad_train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

/tmp/ipython-input-1692717625.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,4.460500
20,3.911800
30,4.226700
40,3.355400
50,4.146400
60,3.823300
70,3.547700
80,4.211800
90,3.635000
100,3.593100


TrainOutput(global_step=4292, training_loss=2.2539800189970145, metrics={'train_runtime': 591.4018, 'train_samples_per_second': 29.016, 'train_steps_per_second': 7.257, 'total_flos': 3362889259192320.0, 'train_loss': 2.2539800189970145, 'epoch': 4.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF")
tokenizer.save_pretrained("/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF")

('/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF/tokenizer_config.json',
 '/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF/special_tokens_map.json',
 '/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF/vocab.txt',
 '/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF/added_tokens.json',
 '/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/CAMELBERT_ON_AQAD_HF"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import torch

eval_set_for_model = aqad_test_dataset.remove_columns(["example_id", "offset_mapping"])
eval_set_for_model.set_format("torch")

loader = DataLoader(
    eval_set_for_model,
    batch_size=args.per_device_eval_batch_size, # Use the batch size defined in args
    shuffle=False,
    collate_fn=default_data_collator
)

all_start_logits = []
all_end_logits = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

with torch.no_grad():
    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        all_start_logits.append(outputs.start_logits.cpu().numpy())
        all_end_logits.append(outputs.end_logits.cpu().numpy())

start_logits = np.concatenate(all_start_logits, axis=0)
end_logits = np.concatenate(all_end_logits, axis=0)

In [ ]:
import collections

example_to_features = collections.defaultdict(list)
for idx, feature in enumerate(aqad_test_dataset):
    example_to_features[feature["example_id"]].append(idx)

In [ ]:
import numpy as np

n_best = 20
max_answer_length = 30
predicted_answers = []

for example in aqad_test_final:
    example_id = example["id"]
    context = example["context"]
    answers = []

    for feature_index in example_to_features[example_id]:
        start_logit = start_logits[feature_index]
        end_logit = end_logits[feature_index]
        offsets = aqad_test_dataset["offset_mapping"][feature_index]

        start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
        end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
        for start_index in start_indexes:
            for end_index in end_indexes:
                # Skip answers that are not fully in the context
                if offsets[start_index] is None or offsets[end_index] is None:
                    continue
                # Skip answers with a length that is either < 0 or > max_answer_length.
                if (
                    end_index < start_index
                    or end_index - start_index + 1 > max_answer_length
                ):
                    continue

                answers.append(
                    {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                )

    best_answer = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})

In [ ]:
metric = evaluate.load("squad")

In [ ]:
theoretical_answers = [
    {"id": ex["id"], "answers": ex["answers"]}
    for ex in aqad_test_final
]

In [ ]:
print(predicted_answers[1])
print(theoretical_answers[1])

{'id': '5968', 'prediction_text': '1695'}
{'id': '5968', 'answers': {'text': ['1695'], 'answer_start': [8]}}


In [ ]:
metric.compute(predictions=predicted_answers, references=theoretical_answers)

{'exact_match': 18.853171155516943, 'f1': 30.08160824145861}

**CAMELBERT ON ARABIC_SQuAD**

In [ ]:
total = len(flat_squad_clean)
print("Total:", total)

# exact counts from the paper:
train_count = 38885
test_count  = 9459

flat_train = flat_squad_clean.select(range(0, train_count))
flat_test  = flat_squad_clean.select(range(train_count, train_count + test_count))

print(len(flat_train), len(flat_test))

Total: 48344
38885 9459


In [ ]:
squad_train_dataset = flat_train.map(
    preprocess_training_examples,
    batched=True,
    remove_columns=flat_train.column_names,
)

Map:   0%|          | 0/38885 [00:00<?, ? examples/s]

In [ ]:
squad_test_dataset = flat_test.map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=flat_test.column_names,
)

Map:   0%|          | 0/9459 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=4,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    report_to="none"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=squad_train_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
)

/tmp/ipython-input-721822465.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Step,Training Loss
10,4.043800
20,4.025200
30,3.049300
40,2.898200
50,3.522100
60,3.644100
70,3.153400
80,2.826200
90,2.783300
100,3.111700


TrainOutput(global_step=39396, training_loss=1.8333244713348402, metrics={'train_runtime': 5357.448, 'train_samples_per_second': 29.412, 'train_steps_per_second': 7.354, 'total_flos': 3.0879789414303744e+16, 'train_loss': 1.8333244713348402, 'epoch': 4.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF")
tokenizer.save_pretrained("/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF")

('/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF/tokenizer_config.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF/special_tokens_map.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF/vocab.txt',
 '/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF/added_tokens.json',
 '/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF/tokenizer.json')

In [ ]:
model_path = "/content/drive/MyDrive/CAMELBERT_ON_ARABIC_SQUAD_HF"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForQuestionAnswering.from_pretrained(model_path)
model.eval()


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [ ]:
from collections import defaultdict

example_to_features = defaultdict(list)

for i, feat in enumerate(squad_test_dataset):
    example_to_features[feat["example_id"]].append(i)


In [ ]:
from torch.utils.data import DataLoader
import numpy as np
import torch

# Version for the MODEL (no offsets)
eval_dataset = squad_test_dataset.remove_columns(["example_id", "offset_mapping"])
eval_dataset.set_format("torch")

loader = DataLoader(
    eval_dataset,
    batch_size=16,
    shuffle=False,
    collate_fn=default_data_collator
)

all_start_logits = []
all_end_logits = []

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

with torch.no_grad():
    for batch in loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        # offset_mapping is NOT inside batch
        outputs = model(**batch)
        all_start_logits.append(outputs.start_logits.cpu().numpy())
        all_end_logits.append(outputs.end_logits.cpu().numpy())

start_logits = np.concatenate(all_start_logits, axis=0)
end_logits = np.concatenate(all_end_logits, axis=0)


In [ ]:
n_best = 20
max_answer_length = 30
predicted_answers = []

for example in flat_test:
    example_id = example["id"]
    context = example["context"]
    answers = []

    # Loop over all feature chunks for this example
    for feat_i in example_to_features[example_id]:
        start_logit = start_logits[feat_i]
        end_logit = end_logits[feat_i]
        offsets = squad_test_dataset[feat_i]["offset_mapping"]

        start_idxs = np.argsort(start_logit)[-n_best:]
        end_idxs   = np.argsort(end_logit)[-n_best:]

        for s in start_idxs:
            for e in end_idxs:
                # Skip offsets that are None (not part of context or special tokens)
                if offsets[s] is None or offsets[e] is None:
                    continue

                # Skip invalid spans
                if e < s or (e - s + 1) > max_answer_length:
                    continue

                start_char = offsets[s][0]
                end_char   = offsets[e][1]

                answers.append({
                    "text": context[start_char:end_char],
                    "logit_score": float(start_logit[s] + end_logit[e])
                })

    # If no answer found → empty prediction
    if len(answers) == 0:
        predicted_answers.append({"id": example_id, "prediction_text": ""})
        continue

    best = max(answers, key=lambda x: x["logit_score"])
    predicted_answers.append({"id": example_id, "prediction_text": best["text"]})

In [ ]:
references = [
    {
        "id": ex["id"],
        "answers": {
            "text": [ans["text"] for ans in ex["answers"]],
            "answer_start": [ans["answer_start"] for ans in ex["answers"]]
        }
    }
    for ex in flat_test
]


In [ ]:
print(predicted_answers[1])
print(references[1])

{'id': '56f89f569b226e1400dd0ce1', 'prediction_text': 'نهر الرون'}
{'id': '56f89f569b226e1400dd0ce1', 'answers': {'text': ['الرون'], 'answer_start': [334]}}


In [ ]:
metric = evaluate.load("squad")
metric.compute(predictions=predicted_answers, references=references)

{'exact_match': 35.87059942911513, 'f1': 51.656385406451335}